In [1]:
import collections.abc
#hyper needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

In [2]:
from os import path
import os
import numpy as np
import re
#import pandas as pd
import pickle
import mne
import matplotlib.pyplot as plt

In [8]:
from dn3.configuratron import ExperimentConfig
from dn3.data.dataset import Thinker, RawTorchRecording
from dn3.trainable.processes import StandardClassification

from dn3_ext import BENDRClassification, LinearHeadBENDR
import torch
import tqdm
import utils
import argparse

# Since we are doing a lot of loading, this is nice to suppress some tedious information
import mne

mne.set_log_level(False)

In [9]:
parser = argparse.ArgumentParser(description="Fine-tunes BENDER models.")
# parser.add_argument('-data', choices=utils.CUSTOM_LOADERS)
parser.add_argument('--ds-config', default="configs/downstream.yml", help="The DN3 config file to use.")
parser.add_argument('--metrics-config', default="configs/metrics.yml", help="Where the listings for config "
                                                                                    "metrics are stored.")
parser.add_argument('--subject-specific', action='store_true', help="Fine-tune on target subject alone.")
parser.add_argument('--mdl', action='store_true', help="Fine-tune on target subject using all extra data.")
parser.add_argument('--freeze-encoder', action='store_true', help="Whether to keep the encoder stage frozen. "
                                                                          "Will only be done if not randomly initialized.")
parser.add_argument('--random-init', action='store_true', help='Randomly initialized BENDR for comparison.')
parser.add_argument('--multi-gpu', action='store_true', help='Distribute BENDR over multiple GPUs')
parser.add_argument('--num-workers', default=4, type=int, help='Number of dataloader workers.')
parser.add_argument('--results-filename', default=None, help='What to name the spreadsheet produced with all '
                                                                     'final results.')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--ds-config DS_CONFIG]
                             [--metrics-config METRICS_CONFIG]
                             [--subject-specific] [--mdl] [--freeze-encoder]
                             [--random-init] [--multi-gpu]
                             [--num-workers NUM_WORKERS]
                             [--results-filename RESULTS_FILENAME]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\chedl\AppData\Roaming\jupyter\runtime\kernel-643a1e93-ce5d-40ba-9ef3-f0e565c2c9b7.json


SystemExit: 2

In [10]:
experiment = ExperimentConfig(args.ds_config)
for ds_name, ds in tqdm.tqdm(experiment.datasets.items(), total=len(experiment.datasets.items()),
                        desc='Datasets'):
    added_metrics, retain_best, _ = utils.get_ds_added_metrics(ds_name, "configs/metrics.yml")
    for fold, (training, validation, test) in enumerate(tqdm.tqdm(utils.get_lmoso_iterator(ds_name, ds))):
        #tqdm.tqdm.write(torch.cuda.memory_summary())
        #dummy_tensor = torch.tensor([1, 0, 1, 1, 0, 0, 1, 1])
        #training = training + (dummy_tensor,)
        model = BENDRClassification.from_dataset(training)
        if not args.random_init:
            model.load_pretrained_modules(experiment.encoder_weights, experiment.context_weights,
                                            freeze_encoder=args.freeze_encoder)
        process = StandardClassification(model, metrics=added_metrics)
        process.set_optimizer(torch.optim.Adam(process.parameters(), ds.lr, weight_decay=0.01))

        # Fit everything
        process.fit(training_dataset=training, validation_dataset=validation, warmup_frac=0.1,
                    retain_best=retain_best, pin_memory=False)

NameError: name 'args' is not defined

In [13]:
experiment = ExperimentConfig('C:\\Users\\chedl\\OneDrive\\Bureau\\EMSE\\3A\\PE\\PE\\configs\\downstream.yml')
for ds_name, ds in tqdm.tqdm(experiment.datasets.items(), total=len(experiment.datasets.items()),
                desc='Datasets'):
    added_metrics, retain_best, _ = utils.get_ds_added_metrics(ds_name, "configs/metrics.yml")
    for fold, (training, validation, test) in enumerate(tqdm.tqdm(utils.get_lmoso_iterator(ds_name, ds))):
        #tqdm.tqdm.write(torch.cuda.memory_summary())
        #dummy_tensor = torch.tensor([1, 0, 1, 1, 0, 0, 1, 1])
        #training = training + (dummy_tensor,)
        model = BENDRClassification.from_dataset(training)
        model.load_pretrained_modules(experiment.encoder_weights, experiment.context_weights)
        process = StandardClassification(model, metrics=added_metrics)
        process.set_optimizer(torch.optim.Adam(process.parameters(), ds.lr, weight_decay=0.01))

        # Fit everything
        process.fit(training_dataset=training, validation_dataset=validation, warmup_frac=0.1,
                retain_best=retain_best, pin_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: 'configs/downstream_datasets.yml'